# Agentic AI Knowledge Graph - Test Notebook

This notebook tests the Knowledge Graph schema and queries.

**Schema**: Principle -> Method -> Implementation -> StandardVersion

## 1. Setup & Connection

In [1]:
import sys
sys.path.insert(0, '..')

import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수 읽기
URI = os.getenv("NEO4J_URI")
USERNAME = os.getenv("NEO4J_USERNAME")
PASSWORD = os.getenv("NEO4J_PASSWORD")
DATABASE = os.getenv("NEO4J_DATABASE")


from src.graph import Neo4jClient

# Create client (uses settings from .env)
client = Neo4jClient()
client.connect()
print("Connected to Neo4j!")

Connected to Neo4j!


## 2. Initialize Database (Optional)

Run this cell to clear and reload seed data. **Skip if data already exists.**

In [ ]:
# Uncomment to initialize (clears existing data!)
# stats = client.initialize(clear_first=True)
# print(f"Initialized: {stats.total_nodes} nodes, {stats.total_relationships} relationships")

Clearing all data from database!


Initialized: 61 nodes, 60 relationships


## 3. Database Statistics

In [3]:
stats = client.get_stats()

print(f"Total Nodes: {stats.total_nodes}")
print(f"Total Relationships: {stats.total_relationships}")
print("\nNodes by Label:")
for label, count in sorted(stats.nodes_by_label.items()):
    print(f"  {label}: {count}")
print("\nRelationships by Type:")
for rel_type, count in sorted(stats.relationships_by_type.items()):
    print(f"  {rel_type}: {count}")

Total Nodes: 61
Total Relationships: 60

Nodes by Label:
  Document: 3
  Implementation: 16
  Method: 25
  Principle: 11
  Standard: 3
  StandardVersion: 3

Relationships by Type:
  ADDRESSES: 37
  COMPLIES_WITH: 2
  HAS_VERSION: 3
  IMPLEMENTS: 10
  INTEGRATES_WITH: 3
  PROPOSES: 3
  USES: 2


## 4. Principle -> Method -> Implementation Paths

In [4]:
paths = client.get_principle_method_impl_paths()

print(f"Found {len(paths)} paths\n")
for path in paths[:10]:
    impls = ", ".join(path['implementations'][:3])
    if len(path['implementations']) > 3:
        impls += f" (+{len(path['implementations'])-3})"
    print(f"{path['principle']} <- {path['method']} <- [{impls}]")

Found 12 paths

Grounding <- GraphRAG <- [microsoft/graphrag]
Grounding <- Retrieval-Augmented Generation <- [LangChain, LlamaIndex]
Memory <- MemGPT <- [Mem0]
Memory <- Temporal KG Memory <- [Zep]
Multi-Agent Collaboration <- CAMEL <- [AutoGen]
Multi-Agent Collaboration <- MetaGPT <- [CrewAI]
Planning <- MemGPT <- [Mem0]
Planning <- MetaGPT <- [CrewAI]
Planning <- Workflow Graph Orchestration <- [LangGraph]
Reasoning <- ReAct <- [LangChain, LangGraph]


## 5. Principles Coverage

In [5]:
coverage = client.get_principles_coverage()

print("Principle Coverage (Methods / Implementations):\n")
for row in coverage:
    bar_m = '#' * row['method_count']
    bar_i = '*' * row['impl_count']
    print(f"{row['principle']:25} | M:{row['method_count']:2} {bar_m}")
    print(f"{'':25} | I:{row['impl_count']:2} {bar_i}")

Principle Coverage (Methods / Implementations):

Planning                  | M: 7 #######
                          | I: 3 ***
Reasoning                 | M: 7 #######
                          | I: 2 **
Reflection                | M: 5 #####
                          | I: 0 
Grounding                 | M: 5 #####
                          | I: 3 ***
Memory                    | M: 4 ####
                          | I: 2 **
Tool Use & Action         | M: 4 ####
                          | I: 3 ***
Learning                  | M: 3 ###
                          | I: 0 
Multi-Agent Collaboration | M: 2 ##
                          | I: 2 **
Perception                | M: 0 
                          | I: 0 
Guardrails                | M: 0 
                          | I: 0 
Tracing                   | M: 0 
                          | I: 0 


## 6. Method Family Distribution

In [ ]:
families = client.get_method_family_distribution()

print("Method Family Distribution:\n")
for row in families:
    bar = '#' * row['count']
    print(f"{row['family']:30} | {row['count']:2} {bar}")

## 7. Standard Compliance

In [ ]:
compliance = client.get_standard_compliance()

if compliance:
    print("Standard Compliance:\n")
    for row in compliance:
        print(f"{row['implementation']} -> {row['standard']} v{row['version']}")
        print(f"  Role: {row['role']}, Level: {row['level']}")
else:
    print("No standard compliance relationships found")

## 8. Query Specific Method (ReAct)

In [ ]:
# Get ReAct method details
react = client.get_node("Method", "m:react")
if react:
    print("ReAct Method:")
    print(f"  Name: {react['name']}")
    print(f"  Family: {react['method_family']}")
    print(f"  Type: {react['method_type']}")
    print(f"  Granularity: {react['granularity']}")
    print(f"  Year: {react.get('year_introduced', 'N/A')}")
    print(f"  Maturity: {react['maturity']}")
    print(f"  Description: {react['description'][:100]}...")

# Get implementations
print("\nImplementations of ReAct:")
impls = client.get_implementations_by_method("m:react")
for impl in impls:
    print(f"  - {impl['name']} ({impl['impl_type']}) - {impl['support_level']}")

## 9. Composite Methods

In [ ]:
composites = client.get_composite_methods()

print("Composite Methods:\n")
for c in composites:
    components = " + ".join(c['components'])
    print(f"{c['composite_method']} = {components}")

## 10. Data Quality Validation

In [ ]:
print("Data Quality Checks:\n")

# Orphan methods (not connected to Principle)
orphan_methods = client.get_orphan_methods()
print(f"Methods without ADDRESSES -> Principle: {len(orphan_methods)}")
for m in orphan_methods:
    print(f"  - {m['name']}")

# Orphan implementations (not connected to Method)
orphan_impls = client.get_orphan_implementations()
print(f"\nImplementations without IMPLEMENTS -> Method: {len(orphan_impls)}")
for i in orphan_impls:
    print(f"  - {i['name']}")

# Uncovered principles (no methods)
uncovered = client.get_uncovered_principles()
print(f"\nPrinciples with no methods: {len(uncovered)}")
for p in uncovered:
    print(f"  - {p['name']}")

# Methods without paper
no_paper = client.get_methods_without_paper()
print(f"\nMethods without paper/seminal_source: {len(no_paper)}")

## 11. Custom Cypher Query

In [ ]:
# Example: Find all RAG-related methods
query = """
MATCH (m:Method)
WHERE m.method_family = 'retrieval_grounding'
RETURN m.name AS name, m.description AS description
ORDER BY m.name
"""

results = client.run_cypher(query)
print("Retrieval/Grounding Methods:\n")
for r in results:
    print(f"{r['name']}")
    print(f"  {r['description'][:80]}...\n")

## 12. Cleanup

In [ ]:
client.close()
print("Connection closed.")